In [1]:
import pandas as pd
import numpy as np

In [149]:
cbcl = pd.read_csv('cbcl_1_5-2023-07-21.csv')
iq = pd.read_csv("iq-2023-07-21.csv")

basic = pd.read_csv("basic_medical_screening-2023-07-21.csv")
dcdq = pd.read_csv("dcdq-2023-07-21.csv")
vineland = pd.read_csv("vineland-3-2023-07-21.csv")
rbsr = pd.read_csv("rbsr-2023-07-21.csv")
scq = pd.read_csv("scq-2023-07-21.csv")

bg_child = pd.read_csv("background_history_child-2023-07-21.csv")
bg_sibling = pd.read_csv("background_history_sibling-2023-07-21.csv")

core = pd.read_csv("core_descriptive_variables-2023-07-21.csv")
indi = pd.read_csv("individuals_registration-2023-07-21.csv")

expiq = pd.read_csv("predicted_iq_experimental-2023-07-21.csv")

/tmp/ipykernel_3011828/4199636074.py:5: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  dcdq = pd.read_csv("dcdq-2023-07-21.csv")
/tmp/ipykernel_3011828/4199636074.py:14: DtypeWarning: Columns (22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  core = pd.read_csv("core_descriptive_variables-2023-07-21.csv")
/tmp/ipykernel_3011828/4199636074.py:15: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  indi = pd.read_csv("individuals_registration-2023-07-21.csv")


# Missing Data handling

In [ ]:
# background history



In [110]:
# basic medical screening

paras = ["dev_id",
    "dev_lang_dis",
    "mood_or_anx",
    "neuro_oth_calc",
    "dev_speech",
    "med_cond_birth",
    "med_cond_neuro",
    "med_cond_visaud",
    "mood_anx",
    "mood_soc_anx",
    "attn_behav",
    "behav_adhd",
    "birth_prem"]

basic[paras] = basic[paras].fillna(0)

# Data merging

In [111]:
# modified code to include all parameters for calculating cognitive impairment
# dfs = [iq, vineland, basic, cbcl, core, background, rbsr, dcdq, indi, scq]
dfs = [basic, indi, core, scq, background, rbsr, dcdq, vineland, cbcl, iq, expiq]  # SORTED in descending order
total = dfs[0].copy(deep=True)

suffixes = ['_' + str(i) for i in range(1, len(dfs))]

for df, sfx in zip(dfs[1:], suffixes):
    total = pd.merge(total, df, on='subject_sp_id', how="left", suffixes=(sfx, ''))

In [162]:
total.shape

(192969, 777)

# Filtering

## Validity

In [163]:
validity_paras = []

total_drop_iq_na = total.dropna(subset=[validity_paras])

## FSIQ null values

In [164]:
total_drop_iq_na = total.dropna(subset=['fsiq'])
total_drop_iq_na.shape

(4091, 777)

# Making demographic subsets

In [166]:
less5 = total_drop_iq_na[total_drop_iq_na['age_at_eval_months_3'] < 61]
less5.shape

(1135, 777)

In [167]:
less18 = total_drop_iq_na[total_drop_iq_na['age_at_eval_months_3'] < 217]
less18.shape

(3816, 777)

# Descriptive statistics

In [168]:
less18['asd_verif'].value_counts(dropna=False)

asd_verif
1.0      3643
998.0     107
0.0        66
Name: count, dtype: int64

# Computed deriv_cog_impair

In [124]:
# maybe skip this code and use the available derived_cog_impair parameter from expiq df

def check_cognitive_impairment(row):
    if row['invalid'] != 1:
        if pd.notnull(row['fsiq_score']) or pd.notnull(row['nviq_score']):
            if pd.notnull(row['fsiq_score']) and row['fsiq_score'] < 80:
                return 1
            elif pd.notnull(row['nviq_score']) and row['nviq_score'] < 80:
                return 1
            else:
                return 0
        elif pd.notnull(row['fsiq_ratio']) or pd.notnull(row['nviq_ratio']):
            if pd.notnull(row['fsiq_ratio']) and row['fsiq_ratio'] < 80:
                return 1
            elif pd.notnull(row['nviq_ratio']) and row['nviq_ratio'] < 80:
                return 1
            else:
                return 0
        elif pd.notnull(row['intel_class']):
            if row['intel_class'] in ['profound', 'severe', 'moderate', 'mild', 'border', 'id_nos']:
                return 1
            else:
                return 0
        elif pd.notnull(row['communication_standard']):
            if row['communication_standard'] < 80:
                return 1
            else:
                return 0
        elif pd.notnull(row['cognitive_impairment_latest']):
            if (row['cognitive_impairment_latest'] == 1):
                return 1
            else:
                return 0
        elif pd.notnull(row['age_at_registration_years']) and pd.notnull(row['language_level_latest']):
            if (row['age_at_registration_years'] >= 4) and (row['language_level_latest'] != 'Uses longer sentences of his/her own and is able to tell you something that happened'):
                return 1
            else:
                return 0
        elif pd.notnull(row['cog_age_level']):
            if row['cog_age_level'] == 'signif_below_age' : 
                return 1
            else:
                return 0
        elif pd.notnull(row['cog_test_score']):
            if row['cog_test_score'] in ['24_below', '25_39', '40_54', '55_69', '70_79']:
                return 1
            else:
                return 0
        elif pd.notnull(row['age_at_registration_months']) and pd.notnull(row['motor_able']) and pd.notnull(row['dev_lang']):
            if (row['age_at_registration_months'] >= 60) and (row['motor_able'] != 1) and (row['dev_lang'] == 1):
                return 1
            else:
                return 0
        elif pd.notnull(row['regress_lang_return_y_n']) and pd.notnull(row['language_level_at_enrollment']):
            if (row['regress_lang_return_y_n'] == 4) and (row['language_level_at_enrollment'] != 'Uses longer sentences of his/her own and is able to tell you something that happened'):
                return 1
            else:
                return 0
        elif pd.notnull(row['age_at_eval_years']) and pd.notnull(row['cog_age_equivalent']) and pd.notnull(row['cog_age_equivalent_in_months']):
            if (row['age_at_eval_years'] < 15) and pd.notnull(row['cog_age_equivalent']) and (row['cog_age_equivalent_in_months']/12*100 < 80):
                return 1
            else:
                return 0

# Define the mapping for cog_age_equivalent
cog_age_map = {
    'under_12mos': 6,
    '12_18mos': 15,
    '18_24mos': 21,
    '2_3years': 30,
    '4_5years': 54,
    '6_8years': 84,
    '9_11years': 120,
    '12_or_older': 144
}

# Add a new column 'cog_age_equivalent_in_months' based on 'cog_age_equivalent'
less5.loc[:, 'cog_age_equivalent_in_months'] = less5['cog_age_equivalent'].apply(lambda x: cog_age_map.get(x, 0))

total_drop_iq_na.loc[:, 'cog_age_equivalent_in_months'] = total_drop_iq_na['cog_age_equivalent'].apply(lambda x: cog_age_map.get(x, 0))
less5_drop_iq_na.loc[:, 'cog_age_equivalent_in_months'] = less5_drop_iq_na['cog_age_equivalent'].apply(lambda x: cog_age_map.get(x, 0))
less18_drop_iq_na.loc[:, 'cog_age_equivalent_in_months'] = less18_drop_iq_na['cog_age_equivalent'].apply(lambda x: cog_age_map.get(x, 0))


# create a parameter called deriv_cog_impair in less5 df
less5.loc[:, 'deriv_cog_impair'] = None

total_drop_iq_na.loc[:, 'deriv_cog_impair'] = None
less18_drop_iq_na.loc[:, 'deriv_cog_impair'] = None
less5_drop_iq_na.loc[:, 'deriv_cog_impair'] = None

# Apply the function to each row of the DataFrame
less5.loc[:, 'deriv_cog_impair'] = less5.apply(check_cognitive_impairment, axis=1)

total_drop_iq_na.loc[:, 'deriv_cog_impair'] = total_drop_iq_na.apply(check_cognitive_impairment, axis=1)
less18_drop_iq_na.loc[:, 'deriv_cog_impair'] = less18_drop_iq_na.apply(check_cognitive_impairment, axis=1)
less5_drop_iq_na.loc[:, 'deriv_cog_impair'] = less5_drop_iq_na.apply(check_cognitive_impairment, axis=1)

In [95]:
# older results (without fsiq filtering)
less5['deriv_cog_impair'].value_counts(dropna=False)

deriv_cog_impair
0.0    12366
NaN     8257
1.0     6465
Name: count, dtype: int64

In [161]:
# new filtered results
print("total", total_drop_iq_na['deriv_cog_impair'].value_counts(dropna=False))
print("\nless18", less18_drop_iq_na['deriv_cog_impair'].value_counts(dropna=False))
print("\nless5", less5_drop_iq_na['deriv_cog_impair'].value_counts(dropna=False))

total deriv_cog_impair
0.0    1997
1.0    1915
NaN     179
Name: count, dtype: int64

less18 deriv_cog_impair
0.0    1830
1.0    1811
NaN     175
Name: count, dtype: int64

less5 deriv_cog_impair
1.0    746
0.0    335
NaN     54
Name: count, dtype: int64


# ExpIQ derived_cog_impair extraction

In [172]:
# print("total", total_drop_iq_na['derived_cog_impair'].value_counts(dropna=False))
print("\nless18", less18_drop_iq_na['derived_cog_impair'].value_counts(dropna=False))
print("\nless5", less5_drop_iq_na['derived_cog_impair'].value_counts(dropna=False))


less18 derived_cog_impair
0.0    1773
1.0    1653
NaN     390
Name: count, dtype: int64

less5 derived_cog_impair
1.0    506
NaN    362
0.0    267
Name: count, dtype: int64


## Compare the overlap/matching of the two variables

In [170]:
# Calculate the number of rows where the values in 'derived_cog_impair' and 'deriv_cog_impair' are the same
same_values_count = less18_drop_iq_na['derived_cog_impair'].eq(less18_drop_iq_na['deriv_cog_impair']).sum()

# Calculate the number of rows where the values in 'derived_cog_impair' and 'deriv_cog_impair' are different
diff_values_count = (~less18_drop_iq_na['derived_cog_impair'].eq(less18_drop_iq_na['deriv_cog_impair'])).sum()

# Print the results
print("Number of rows where the values are the same:")
print(same_values_count)

print("\nNumber of rows where the values are different:")
print(diff_values_count)

Number of rows where the values are the same:
3236

Number of rows where the values are different:
580


In [171]:
# Calculate the number of rows where the values in 'derived_cog_impair' and 'deriv_cog_impair' are different
# Using groupby on the two columns to find all possible combinations and their counts
mismatch_combinations = less18_drop_iq_na.groupby(
    ['derived_cog_impair', 'deriv_cog_impair']
).size().reset_index(name='count')

# Filter only mismatched values
mismatch_combinations = mismatch_combinations[
    mismatch_combinations['derived_cog_impair'] != mismatch_combinations['deriv_cog_impair']
]

# Print the results
print("Count of each combination of mismatched values:")
print(mismatch_combinations)

Count of each combination of mismatched values:
   derived_cog_impair  deriv_cog_impair  count
1                 0.0               1.0     11
2                 1.0               0.0     16


**Conclusion**

3405 matching
27 mismatching
580 either both or one value missing, hence mismatched

the parameter we computed (deriv_cog_impair) has less missing values compared to the existing parameter (derived_cog_impair)

# Dataset export

In [173]:
# save CSV files
total_drop_iq_na.to_csv("total_fsiq.csv", index=False)
less18_drop_iq_na.to_csv("less18_fsiq.csv", index=False)
less5_drop_iq_na.to_csv("tless5_fsiq.csv", index=False)

In [174]:
# ref129 = less5.copy()

ref129_total = total_drop_iq_na.copy()
ref129_less18 = less18_drop_iq_na.copy()
ref129_less5 = less5_drop_iq_na.copy()

# minimal dataset (129 parameters)

In [99]:
# extract required parameters

model_para = [
    # 
    "subject_sp_id",
    "asd_5",
    "derived_cog_impair",
    #
    # BACKGROUND --------------
    "age_onset_mos",   # standardisation
    "cog_age_level",   # ENCODING
    "family_hx_asd_mat_aunt_uncle",
    "father_highest_education",
    "fed_self_spoon_age_mos",
    "function_age_level",   # ENCODING
    "hand",
    "intervention_counseling",
    "intervention_medication",
    "intervention_ot_fine_mot",
    "intervention_ot_sensory",
    "intervention_social_skills",
    "intervention_speech_language",
    "language_age_level",   # ENCODING
    "mother_highest_education",
    "onset_concern",
    "plateau_y_n",
    "regress_lang_return_y_n",
    "regress_lang_time_return",
    "regress_lang_y_n",
    "repeat_grade",
    "smiled_age_mos",
    "sped_504",
    "sped_aide",
    "sped_asd_class_ft",
    "sped_birth_to_three",
    "sped_iep_asd",
    "sped_iep_other",
    "sped_ot",
    "sped_other",
    "sped_other_class_ft",
    "sped_other_class_pt",
    "sped_preschool",
    "sped_priv_asd",
    "sped_pt",
    "sped_soc_skills",
    "sped_speech",
    "sped_y_n",
    "support_require",
    # 
    # SCQ -----------
    "q01_phrases",
    "q02_conversation",
    "q04_inappropriate_question",
    "q06_invented_words",
    "q09_expressions_appropriate",
    "q10_hand_tool",
    "q13_interests_intensity",
    "q14_senses",
    "q18_objects_carry",
    "q20_talk_friendly",
    "q21_copy_you",
    "q22_point_things",
    "q23_gestures_wanted",
    "q24_nod_head",
    "q25_shake_head",
    "q26_look_directly",
    "q28_things_interested",
    "q29_share",
    "q30_join_enjoyment",
    "q31_comfort",
    "q32_help_attention",
    "q33_range_expressions",
    "q35_make_believe",
    "q36_same_age",
    "q38_pay_attention",
    # 
    # DCDQ --------------------------
    "q02_catch_ball",
    "q05_run_fast_similar",
    "q06_plan_motor_activity",
    "q09_appropriate_tension_printing_writing",
    "q10_cuts_pictures_shapes",
    "q11_likes_sports_motors_skills",
    "q13_quick_competent_tidying_up",
    "fine_motor_handwriting",
    #
    # basic medical screening --------------
    "dev_id",
    "dev_lang_dis",
    "mood_or_anx",
    "neuro_oth_calc",
    "dev_speech",
    "med_cond_birth",
    "med_cond_neuro",
    "med_cond_visaud",
    "mood_anx",
    "mood_soc_anx",
    "attn_behav",
    "behav_adhd",
    "birth_prem",
    # 
    # individuals registration -------------
    "diagnosis_age",
    "ASD_services",
    "iep_asd",
    "cognitive_impairment_at_enrollment",   # ENCODING
    "language_level_at_enrollment",   # ENCODING
    # 
    # RBSR --------------------
    "q01_whole_body",
    "q03_hand_finger",
    "q07_hits_self_body",
    "q08_hits_self_against_object",
    "q09_hits_self_with_object",
    "q12_rubs",
    "q16_complete",
    "q18_checking",
    "q19_counting",
    "q22_touch_tap",
    "q32_insists_walking",
    "q27_play",
    "q28_communication",
    "q29_things_same_place",
    "q31_becomes_upset",
    "q34_dislikes_changes",
    "q35_insists_door",
    "q36_likes_piece_music",
    "q39_insists_time",
    "q41_strongly_attached",
    "q43_fascination_movement"
]


ref129_total = ref129_total[model_para]
ref129_less18 = ref129_less18[model_para]
ref129_less5 = ref129_less5[model_para]

In [100]:
ref129_total.rename(columns={'asd_5': 'asd'}, inplace=True)
ref129_less18.rename(columns={'asd_5': 'asd'}, inplace=True)
ref129_less5.rename(columns={'asd_5': 'asd'}, inplace=True)

## encoding and elimination of extra parameters

In [101]:
# perform one hot encoding of the dataframe

# Identify categorical columns
categorical_columns = ['cog_age_level', 'father_highest_education', 'function_age_level', 'hand', 'language_age_level',
                   'mother_highest_education', 'onset_concern', 'regress_lang_time_return', 'support_require',
                   'ASD_services', 'iep_asd', 'cognitive_impairment_at_enrollment', 'language_level_at_enrollment']

# One-hot encode categorical columns
ref129_total_encoded = pd.get_dummies(ref129_total, columns=categorical_columns)
ref129_less18_encoded = pd.get_dummies(ref129_less18, columns=categorical_columns)
ref129_less5_encoded = pd.get_dummies(ref129_less5, columns=categorical_columns)

In [102]:
# drop paramters

drop_para = [
    'father_highest_education_associate_degree',
    'father_highest_education_did_not_attend_high_school',
    'father_highest_education_ged_diploma',
    'father_highest_education_graduate_or_professional_degree',
    'father_highest_education_some_high_school',
    'father_highest_education_trade_school',
    'function_age_level_above_age',
    'function_age_level_at_age',
    'function_age_level_slight_below_age',
    'hand_left',
    'hand_not_able',
    'language_age_level_at_age',
    'language_age_level_slight_below_age',
    'mother_highest_education_associate_degree',
    'mother_highest_education_baccalaureate_degree',
    'mother_highest_education_did_not_attend_high_school',
    'mother_highest_education_ged_diploma',
    'mother_highest_education_some_college',
    'mother_highest_education_some_high_school',
    'mother_highest_education_trade_school',
    'onset_concern_change_loss',
    'onset_concern_something_else',
    'onset_concern_unusual_habits',
    'onset_concern_unusual_speech',
    'support_require_substantial',
    'ASD_services_False',
    'iep_asd_False',
    'cognitive_impairment_at_enrollment_False',
    'language_level_at_enrollment_No words/does not speak'
]


# ref129_dropped = ref129_encoded.drop(columns=drop_para) 

ref129_total_encoded = ref129_total_encoded.drop(columns=drop_para) 
ref129_less18_encoded = ref129_less18_encoded.drop(columns=drop_para) 
ref129_less5_encoded = ref129_less5_encoded.drop(columns=drop_para) 

## Boolean to 0/1 replacement

In [103]:
# List of parameters to replace
parameters_to_replace = [
    'cog_age_level_above_age', 'cog_age_level_at_age', 'cog_age_level_signif_below_age',
    'cog_age_level_slight_below_age', 'father_highest_education_baccalaureate_degree',
    'father_highest_education_high_school_graduate', 'father_highest_education_some_college',
    'function_age_level_signif_below_age', 'hand_ambi', 'hand_right',
    'language_age_level_above_age', 'language_age_level_signif_below_age',
    'mother_highest_education_graduate_or_professional_degree',
    'mother_highest_education_high_school_graduate', 'onset_concern_late_speech',
    'onset_concern_late_walk', 'onset_concern_mood', 'onset_concern_social',
    'regress_lang_time_return_5_or_more_yr', 'regress_lang_time_return_6_to_12_mo',
    'regress_lang_time_return_bt_1_to_2_yr', 'regress_lang_time_return_bt_2_to_3_yr',
    'regress_lang_time_return_bt_3_to_4_yr', 'regress_lang_time_return_bt_4_to_5_yr',
    'regress_lang_time_return_less_than_6_mo', 'support_require_minimal_no_support',
    'support_require_some_support', 'support_require_very_substantial', 'ASD_services_True',
    'iep_asd_True', 'cognitive_impairment_at_enrollment_True',
    'language_level_at_enrollment_Combines 3 words together into short sentences',
    'language_level_at_enrollment_Uses longer sentences of his/her own and is able to tell you something that happened',
    'language_level_at_enrollment_Uses single words meaningfully (for example, to request)'
]

# Iterate through the list of parameters and perform replacement
for param in parameters_to_replace:
    ref129_total_encoded[param] = ref129_total_encoded[param].map({True: 1, False: 0})
    ref129_less18_encoded[param] = ref129_less18_encoded[param].map({True: 1, False: 0})
    ref129_less5_encoded[param] = ref129_less5_encoded[param].map({True: 1, False: 0})

# Missing values evaluation

In [153]:
nan_counts = ref129_less18_encoded.isnull().sum()
more_na = nan_counts[nan_counts > 0]

more_na

asd                              918
deriv_cog_impair                 175
age_onset_mos                    934
family_hx_asd_mat_aunt_uncle    3647
fed_self_spoon_age_mos          1046
                                ... 
q35_insists_door                 980
q36_likes_piece_music            975
q39_insists_time                 975
q41_strongly_attached            977
q43_fascination_movement         971
Length: 87, dtype: int64

In [154]:
i = 1
for item in more_na.index:
    print(i, item)
    i = i+1

1 asd
2 deriv_cog_impair
3 age_onset_mos
4 family_hx_asd_mat_aunt_uncle
5 fed_self_spoon_age_mos
6 intervention_counseling
7 intervention_medication
8 intervention_ot_fine_mot
9 intervention_ot_sensory
10 intervention_social_skills
11 intervention_speech_language
12 plateau_y_n
13 regress_lang_return_y_n
14 regress_lang_y_n
15 repeat_grade
16 smiled_age_mos
17 sped_504
18 sped_aide
19 sped_asd_class_ft
20 sped_birth_to_three
21 sped_iep_asd
22 sped_iep_other
23 sped_ot
24 sped_other
25 sped_other_class_ft
26 sped_other_class_pt
27 sped_preschool
28 sped_priv_asd
29 sped_pt
30 sped_soc_skills
31 sped_speech
32 sped_y_n
33 q01_phrases
34 q02_conversation
35 q04_inappropriate_question
36 q06_invented_words
37 q09_expressions_appropriate
38 q10_hand_tool
39 q13_interests_intensity
40 q14_senses
41 q18_objects_carry
42 q20_talk_friendly
43 q21_copy_you
44 q22_point_things
45 q23_gestures_wanted
46 q24_nod_head
47 q25_shake_head
48 q26_look_directly
49 q28_things_interested
50 q29_share
51 q

- lots of SCQ and RBSR parameters are missing in a huge portion of dataset
- basic medical screening parameters can be imputed with 0 

In [155]:
# check missing parameters for RBSR

paras = ["q01_whole_body",
    "q03_hand_finger",
    "q07_hits_self_body",
    "q08_hits_self_against_object",
    "q09_hits_self_with_object",
    "q12_rubs",
    "q16_complete",
    "q18_checking",
    "q19_counting",
    "q22_touch_tap",
    "q32_insists_walking",
    "q27_play",
    "q28_communication",
    "q29_things_same_place",
    "q31_becomes_upset",
    "q34_dislikes_changes",
    "q35_insists_door",
    "q36_likes_piece_music",
    "q39_insists_time",
    "q41_strongly_attached",
    "q43_fascination_movement"]

ref129_less18_encoded[paras].isnull().sum()

q01_whole_body                  983
q03_hand_finger                 978
q07_hits_self_body              973
q08_hits_self_against_object    978
q09_hits_self_with_object       976
q12_rubs                        986
q16_complete                    973
q18_checking                    983
q19_counting                    977
q22_touch_tap                   975
q32_insists_walking             979
q27_play                        973
q28_communication               971
q29_things_same_place           973
q31_becomes_upset               977
q34_dislikes_changes            975
q35_insists_door                980
q36_likes_piece_music           975
q39_insists_time                975
q41_strongly_attached           977
q43_fascination_movement        971
dtype: int64

In [136]:
# check missing parameters for SCQ

paras = ["q01_phrases",
    "q02_conversation",
    "q04_inappropriate_question",
    "q06_invented_words",
    "q09_expressions_appropriate",
    "q10_hand_tool",
    "q13_interests_intensity",
    "q14_senses",
    "q18_objects_carry",
    "q20_talk_friendly",
    "q21_copy_you",
    "q22_point_things",
    "q23_gestures_wanted",
    "q24_nod_head",
    "q25_shake_head",
    "q26_look_directly",
    "q28_things_interested",
    "q29_share",
    "q30_join_enjoyment",
    "q31_comfort",
    "q32_help_attention",
    "q33_range_expressions",
    "q35_make_believe",
    "q36_same_age",
    "q38_pay_attention"]

ref129_total_encoded[paras].isnull().sum()

q01_phrases                     419
q02_conversation               1106
q04_inappropriate_question     1112
q06_invented_words             1110
q09_expressions_appropriate     416
q10_hand_tool                   413
q13_interests_intensity         416
q14_senses                      415
q18_objects_carry               410
q20_talk_friendly               421
q21_copy_you                    432
q22_point_things                438
q23_gestures_wanted             429
q24_nod_head                    434
q25_shake_head                  427
q26_look_directly               429
q28_things_interested           434
q29_share                       429
q30_join_enjoyment              432
q31_comfort                     429
q32_help_attention              433
q33_range_expressions           437
q35_make_believe                429
q36_same_age                    426
q38_pay_attention               428
dtype: int64

In [156]:
# check missing parameters for background

paras = [# BACKGROUND --------------
    "age_onset_mos",   # standardisation
    #"cog_age_level",   # ENCODING
    "family_hx_asd_mat_aunt_uncle",
    #"father_highest_education",   # ENCODING
    "fed_self_spoon_age_mos",
    #"function_age_level",   # ENCODING
    #"hand",   # ENCODING
    "intervention_counseling",
    "intervention_medication",
    "intervention_ot_fine_mot",
    "intervention_ot_sensory",
    "intervention_social_skills",
    "intervention_speech_language",
    #"language_age_level",   # ENCODING
    #"mother_highest_education",
    #"onset_concern",
    "plateau_y_n",
    "regress_lang_return_y_n",
    #"regress_lang_time_return",
    "regress_lang_y_n",
    "repeat_grade",
    "smiled_age_mos",
    "sped_504",
    "sped_aide",
    "sped_asd_class_ft",
    "sped_birth_to_three",
    "sped_iep_asd",
    "sped_iep_other",
    "sped_ot",
    "sped_other",
    "sped_other_class_ft",
    "sped_other_class_pt",
    "sped_preschool",
    "sped_priv_asd",
    "sped_pt",
    "sped_soc_skills",
    "sped_speech",
    "sped_y_n",
    #"support_require"
    ]

ref129_less18_encoded[paras].isnull().sum()
# ref129_less5_encoded[paras].isnull().sum()

age_onset_mos                    934
family_hx_asd_mat_aunt_uncle    3647
fed_self_spoon_age_mos          1046
intervention_counseling         2893
intervention_medication         2568
intervention_ot_fine_mot        2131
intervention_ot_sensory         2497
intervention_social_skills      3079
intervention_speech_language    1826
plateau_y_n                      928
regress_lang_return_y_n         2960
regress_lang_y_n                 930
repeat_grade                     982
smiled_age_mos                   991
sped_504                        3550
sped_aide                       2966
sped_asd_class_ft               3448
sped_birth_to_three             2411
sped_iep_asd                    2001
sped_iep_other                  2390
sped_ot                         1811
sped_other                      3615
sped_other_class_ft             3634
sped_other_class_pt             3639
sped_preschool                  2005
sped_priv_asd                   3649
sped_pt                         3016
s

- RBSR : most para missing in 1100_ entries
- SCQ : most parameters missing in 410+ entries
    - "q02_conversation", "q04_inappropriate_question", "q06_invented_words" missing in 1100+ entries
- background history
    - cannot be imputed:
    age_onset_mos, fed_self_spoon_age_mos, smiled_age_mos
    plateau_y_n, regress_lang_return_y_n, regress_lang_y_n, repeat_grade
    
**Criteria parameters (least missing)**

- RBSR => q28_communication
- SCQ => q18_objects_carry

## 

In [140]:
# Define the parameters
param1 = 'q28_communication'
param2 = 'q18_objects_carry'

# Filter rows with non-null values in both parameters
non_null_overlap = ref129_total_encoded.dropna(subset=['q28_communication', 'q18_objects_carry'])

# Count the number of rows where neither of the two parameters are null
non_null_count = non_null_overlap.shape[0]

# Print the count
print(f"Number of entries where neither '{param1}' nor '{param2}' are null: {non_null_count}")

Number of entries where neither 'q28_communication' nor 'q18_objects_carry' are null: 2816


In [141]:
# which parameter has the most missing values in ref129_total_encoded?

# Calculate the number of missing values in each column
missing_values_count = ref129_total_encoded.isnull().sum()

# Find the parameter (column) with the most missing values
parameter_with_most_missing = missing_values_count.idxmax()

# Get the count of missing values in that parameter
max_missing_count = missing_values_count[parameter_with_most_missing]

# Print the result
print(f"The parameter with the most missing values is '{parameter_with_most_missing}' with {max_missing_count} missing values.")

The parameter with the most missing values is 'family_hx_asd_mat_aunt_uncle' with 3919 missing values.


## Eliminate entries with too many missing values

## Export final ref129 file

In [49]:
# save CSV files
# ref129_dropped.to_csv("ref129_2.csv", index=False)
ref_copy.to_csv("ref129_latest.csv", index=False)